<a href="https://colab.research.google.com/github/oakeshott/21SS_CSPTP-based_ILP/blob/master/21SummerSeminarTasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5G通信基盤を支えるネットワーク仮想化技術によるネットワークの最適設計

## 目的

[前回の演習](https://colab.research.google.com/drive/1ab9VKYQtsQ7VMGVRuzrq6lgtcfzo6zsK?usp=sharing)より，SPTPとサービスチェイニングの関連性について理解しました．
この演習では，スケーラビリティの観点からSPTPに基づくサービスチェイニングの複雑性を理解することを目的としています．

## 課題

スケーラビリティの観点から，2つの課題に取り組んでもらう予定です．
1. 物理ノード数に対する処理性能
2. サービスチェイン要求数に対する処理性能

なお，実験を行いやすくするために`solve()`関数を用意しました．
これは前回作成したSPTPに基づくサービスチェイニングを行うILPを解く関数です．
  - 入力: データセットが存在するディレクトリのパス
  - 出力: 状態(status)，最適値(objective value), 実行時間(execution time)

例えば，以下のようなプログラムを実行すると，次のような出力が得られます．
```python
path = "/content/21SS_CSPTP-based_ILP/dataset/nsfnetbw"
status, objective_value, execution_time = solve(path)
print(status, objective_value, execution_time)
Optimal 121.44999999999996 0.06714981600089232
```



In [21]:
!pip install pulp networkx matplotlib pandas

In [34]:
!rm -rf /content/21SS_CSPTP-based_ILP/
!git clone https://github.com/oakeshott/21SS_CSPTP-based_ILP

Cloning into '21SS_CSPTP-based_ILP'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 166 (delta 112), reused 157 (delta 106), pack-reused 0
Receiving objects: 100% (166/166), 10.00 MiB | 20.82 MiB/s, done.
Resolving deltas: 100% (112/112), done.


In [35]:
import pulp
import networkx as nx
import os
import matplotlib.pyplot as plt
import json
import time
import pandas as pd

In [36]:
def solve(path="/content/21SS_CSPTP-based_ILP/dataset/nsfnetbw"):
    filename = os.path.join(path, "network.gml")
    physical_network = nx.read_gml(filename, label="id")
    physical_node_size = len(physical_network.nodes())

    filename = os.path.join(path, "service_chain_requirements.json")
    with open(filename) as f:
        connections = json.load(f)
    for connection in connections:
        connection['required_processing_func'] = {int(k): v for k, v in connection['required_processing_func'].items()}

    filename = os.path.join(path, "function_placement.json")
    with open(filename) as f:
      function_placement = json.load(f)
    placement = function_placement["placement"]
    F = set(function_placement['functions'])

    imaginary_links = pd.DataFrame.from_dict(placement)
    imaginary_nodes = {f: f+physical_node_size for f in F}

    G = physical_network.copy()
    for func, node in imaginary_nodes.items():
        G.add_node(node, lng=-(func+1)*8 - 70, lat=60, delay=0)
    imaginary_links.apply(lambda x: G.add_edge(x.physical_node, imaginary_nodes[x.function], delay=0), axis=1)
    imaginary_links.apply(lambda x: G.add_edge(imaginary_nodes[x.function], x.physical_node, delay=x.delay * 1e-3), axis=1) # d_{i,j}^{func}

    a = []
    b = []
    for c in connections:
        R_c = c['request']
        K_c = len(R_c)
        o_c = c['origin']
        d_c = c['destination']
        a_ck = []
        b_ck = []
        for k in range(K_c+1):
            if k == 0:
                a_ck.append(o_c)
            if k == K_c:
                b_ck.append(d_c)
            else:
                a_ck.append(imaginary_nodes[R_c[k]])
                b_ck.append(imaginary_nodes[R_c[k]])
        a.append(a_ck)
        b.append(b_ck)

    # formualte ILP
    m = pulp.LpProblem("SPTP-based-ILP", pulp.LpMinimize)

    x = {}
    for c, connection in enumerate(connections):
        K_c = len(connection['request'])
        for k in range(K_c+1):
            for i, j in G.edges():
                x[i,j,c,k] = pulp.LpVariable("x_{:},{:},{:},{:}".format(i, j, c, k), 0, 1, pulp.LpBinary)

    m += pulp.lpSum(G[i][j]["delay"] * pulp.lpSum(x[i,j,c,k]           for c in range(len(connections)) for k in range(len(connections[c]['request'])+1) ) for i, j in physical_network.edges()) \
            + pulp.lpSum(G.nodes()[v]["delay"] * pulp.lpSum(x[v,j,c,k] for c in range(len(connections)) for k in range(len(connections[c]['request'])+1) ) for v, j in physical_network.edges()) \
            + pulp.lpSum(G[v_f][v]["delay"] * pulp.lpSum(x[v_f,v,c,k]  for c in range(len(connections)) for k in range(len(connections[c]['request'])+1) ) for v_f, v in G.edges() if v_f in imaginary_nodes.values()), "Objective function"

    for c, connection in enumerate(connections):
        K_c = len(connection['request'])
        for k in range(K_c + 1):
            for i in G.nodes():
                if i == a[c][k]:
                    m += pulp.lpSum(x[i, j, c, k] for j in G.neighbors(i)) == 1, f"flow_constraint_c{c}_k{k}_i{i}"
                elif i == b[c][k]:
                    m += pulp.lpSum(x[j, i, c, k] for j in G.neighbors(i) ) == 1, f"flow_constraint_c{c}_k{k}_i{i}"
                else:
                    m += pulp.lpSum(x[i, j, c, k] for j in G.neighbors(i)) == pulp.lpSum(x[j, i, c, k] for j in G.neighbors(i)), f"flow_constraint_c{c}_k{k}_i{i}"

    for c, connection in enumerate(connections):
        K_c = len(connection['request'])
        for k in range(K_c):
            v_f_ck = b[c][k]
            for i in G.neighbors(v_f_ck):
                m += x[i, v_f_ck, c, k] == x[v_f_ck, i, c, k+1], "connectivity_of_subpath_constraint_i{:}j{:}c{:}k{:}".format(i, v_f_ck, c, k)

    for c, connection in enumerate(connections):
        K_c = len(connection['request'])
        for k in range(K_c+1):
            for i, v_f in G.edges():
                if v_f in imaginary_nodes.values() and not v_f is b[c][k]:
                    m += x[i, v_f, c, k] == 0, "infeasible_link_i{:}j{:}k{:}c{:}".format(i, v_f, k, c)

    for i, j in physical_network.edges():
        m += pulp.lpSum(connection['required_bandwidth'] * pulp.lpSum(x[i, j, c, k] for k in range(len(connection['request'])+1)) for c, connection in enumerate(connections)) <= physical_network[i][j]["bandwidth"], "physical_link_capacity_i{:}j{:}".format(i, j)

    for v in physical_network.nodes():
        m += pulp.lpSum(connection["required_processing_node"] * pulp.lpSum(x[v,j,c,k] for j in physical_network.neighbors(v) for k in range(len(connection['request'])+1)) + \
                pulp.lpSum(connection["required_processing_func"][v_f-physical_node_size] * pulp.lpSum(x[v_f,v,c,k] for k in range(len(connection['request'])+1)) for v_f in G.neighbors(v) if v_f in imaginary_nodes.values() and v_f-physical_node_size in connection['request']) for c, connection in enumerate(connections)) \
                <= physical_network.nodes()[v]["capacity"], "processing_capacity_constraint_v{:}".format(v)

    for c, connection in enumerate(connections):
        K_c = len(connection['request'])
        o = connection['origin']
        d = connection['destination']
        m += pulp.lpSum(x[i,o,c,0] for i in G.neighbors(o)) == 0, "constraint_7_w{:}".format(c)
        m += pulp.lpSum(x[d,i,c,len(connection['request'])] for i in G.neighbors(d)) == 0, "constraint_8_w{:}".format(c)

    start = time.perf_counter()
    status = m.solve()
    end = time.perf_counter()
    elapsed = end - start

    return pulp.LpStatus[status], pulp.value(m.objective), elapsed


In [48]:
path = "/content/21SS_CSPTP-based_ILP/dataset/nsfnetbw"
status, objective_value, execution_time = solve(path)
print(status, objective_value, execution_time)

Optimal 121.44999999999996 0.06714981600089232


# ノード数に対する処理性能の変化
まず，SPTPに基づくILPのスケーラビリティを確認するために，ノード数に対する処理性能を調査してもらいます．  
ノード数100から1000までのファイル群を入力として，ノード数の増加に伴い，実行時間がどのように推移するのかを確認してください．  
入力ファイルに関しては，下記のディレクトリに存在します．  
`/content/21SS_CSPTP-based_ILP/dataset/nrof_nodes{n}`  
ここで`{n}`はノード数を表しています．

なお，結果に関しては，図などを利用して表示してください．
その際に，[matplotlib](https://matplotlib.org/)や[pandas](https://pandas.pydata.org/docs/)を利用してもらうことになるかと思います．


In [ ]:
!ls /content/21SS_CSPTP-based_ILP/dataset/nrof_nodes*

# サービスチェイン要求数に対する処理性能の変化
まず，SPTPに基づくILPのスケーラビリティを確認するために，サービスチェイン要求数に対する処理性能を調査してもらいます．  
ノード数100から1000までのファイル群を入力として，サービスチェイン要求数の増加に伴い，実行時間がどのように推移するのかを確認してください．  
入力ファイルに関しては，下記のディレクトリに存在します．  
`/content/21SS_CSPTP-based_ILP/dataset/nrof_connections{c}`  
ここで`{c}`はサービスチェイン要求数を表しています．

なお，結果に関しては同様に，図などを利用して表示してください．
その際に，[matplotlib](https://matplotlib.org/)や[pandas](https://pandas.pydata.org/docs/)を利用してもらうことになるかと思います．


In [ ]:
!ls /content/21SS_CSPTP-based_ILP/dataset/nrof_connections*

# 課題3: 運用面を考慮したサービスチェイニング (時間があれば)

課題1，2より，スケーラビリティの観点から，SPTPに基づくサービスチェイニングのILPは十分な計算量が必要となることを確認しました．
一般に解の最適性と計算時間にはトレードオフ関係にあることがいえます．
計算時間の増加はユーザに迅速なサービス提供ができなくなり，QoSの低下に繋がります．
これは運用面においてもシステムが機能しなくなり，管理コストが増大することが懸念されます．
ネットワークに接続するデバイス数の増加や多様化するネットワークサービスに伴い，ネットワークは複雑化する中で迅速かつ最適なサービスチェイニングを実現する方法を検討してください．（アイデアベースのものでも構いません．）